In [1]:
import pandas as pd
import numpy as np 
import os 
from pathlib import Path

In [3]:
# Team Info 
womens_teams = pd.read_csv("../../data/WTeams.csv")

# Season Info 
womens_seasons = pd.read_csv("../../data/Wseasons.csv")

# Seed Info 
womens_tourney_seeds = pd.read_csv("../../data/WNCAATourneySeeds.csv")

# Regular Season Data - Each record: game
# Regular Seasons Games - All games before day num 132 
womens_reg_season_data = pd.read_csv("../../data/WRegularSeasonCompactResults.csv")

# Tourney Data 
womens_tourney_data = pd.read_csv("../../data/WNCAATourneyCompactResults.csv")

# Regular Season Box Scores 
womens_reg_season_box_scores = pd.read_csv("../../data/WRegularSeasonDetailedResults.csv")

# Tournament Box Scores 
womens_tourney_box_scores = pd.read_csv("../../data/WNCAATourneyDetailedResults.csv")

# Conferences 
conferences = pd.read_csv("../../data/Conferences.csv")
womens_conferences = pd.read_csv("../../data/WTeamConferences.csv")

# Alternate Spellings 
womens_spellings = pd.read_csv("../../data/WTeamSpellings.csv", encoding='ISO-8859-1')

#Tourney Slots 
womens_tourney_slots = pd.read_csv("../../data/WNCAATourneySlots.csv")

# Conference Tourney 
womens_conf_tourney = pd.read_csv("../../data/WConferenceTourneyGames.csv")

# Geography  
cities = pd.read_csv("../../data/Cities.csv")
womens_cities = pd.read_csv("../../data/WGameCities.csv")

# Sample Submission
sample = pd.read_csv("../../data/SampleSubmissionStage1.csv")

In [4]:
### Calculate summary statistics about each mens basketball team for each season 

winning_teams = womens_reg_season_box_scores.groupby(['Season', 'WTeamID']).agg({
    'WScore': 'sum', 
    'LScore': 'sum',     
    'WFGM': 'sum', 
    'WFGA': 'sum',
    'WFGM3': 'sum',
    'WFGA3': 'sum',
    'WFTM': 'sum',
    'WFTA': 'sum',
    'WOR': 'sum', 
    'WDR': 'sum', 
    'WAst': 'sum', 
    'WTO': 'sum', 
    'LFGM': 'sum', 
    'LFGA': 'sum', 
    'LFGM3': 'sum',
    'LFGA3': 'sum',
    'LFTM': 'sum', 
    'LFTA': 'sum', 
    'LOR': 'sum', 
    'LDR': 'sum', 
    'LTO': 'sum'
}).reset_index() 

winning_teams['Wins'] = womens_reg_season_box_scores.groupby(['Season', 'WTeamID']).size().values

losing_teams = womens_reg_season_box_scores.groupby(['Season', 'LTeamID']).agg({  
    'LScore': 'sum',  
    'WScore': 'sum', 
    'LFGM': 'sum', 
    'LFGA': 'sum', 
    'LFGM3': 'sum',
    'LFGA3': 'sum',
    'LFTM': 'sum', 
    'LFTA': 'sum', 
    'LOR': 'sum', 
    'LDR': 'sum', 
    'LTO': 'sum',
    'LAst': 'sum',  
    'WFGM': 'sum', 
    'WFGA': 'sum',
    'WFGM3': 'sum',
    'WFGA3': 'sum',
    'WFTM': 'sum',
    'WFTA': 'sum',
    'WOR': 'sum', 
    'WDR': 'sum', 
    'WTO': 'sum'
}).reset_index()

losing_teams['Losses'] = womens_reg_season_box_scores.groupby(['Season', 'LTeamID']).size().values

merged_df = pd.merge(winning_teams, losing_teams, left_on=['Season', 'WTeamID'], right_on = ['Season', 'LTeamID'], how='outer', suffixes=('_1', '_2'))

# Combine the Stats for Winning and Losing games 
merged_df['Points_Scored'] = merged_df['WScore_1'] + merged_df['LScore_2']
merged_df['FG_Made'] = merged_df['WFGM_1'] + merged_df['LFGM_2']
merged_df['FG_Atts'] = merged_df['WFGA_1'] + merged_df['LFGA_2']
merged_df['Threes_Made'] = merged_df['WFGM3_1'] + merged_df['LFGM3_2']
merged_df['Threes_Attempted'] = merged_df['WFGA3_1'] + merged_df['LFGA3_2']
merged_df['FreeThrows_Made'] = merged_df['WFTM_1'] + merged_df['LFTM_2']
merged_df['FreeThrows_Attempted'] = merged_df['WFTA_1'] + merged_df['LFTA_2']
merged_df['Offensive_Rebounds'] = merged_df['WOR_1'] + merged_df['LOR_2']
merged_df['Defensive_Rebounds'] = merged_df['WDR_1'] + merged_df['LDR_2']
merged_df['Assists'] = merged_df['WAst'] + merged_df['LAst']
merged_df['Turnovers'] = merged_df['WTO_1'] + merged_df['LTO_2']
merged_df['Points_Allowed'] = merged_df['LScore_1'] + merged_df['WScore_2']
merged_df['Opposing_FG_Made'] = merged_df['LFGM_1'] + merged_df['WFGM_2']
merged_df['Opposing_FG_Atts'] = merged_df['LFGA_1'] + merged_df['WFGA_2']
merged_df['Opposing_Threes_Made'] = merged_df['LFGM3_1'] + merged_df['WFGM3_2']
merged_df['Opposing_Threes_Attempted'] = merged_df['LFGA3_1'] + merged_df['WFGA3_2']
merged_df['Opposing_FreeThrows_Made'] = merged_df['LFTM_1'] + merged_df['WFTM_2']
merged_df['Opposing_FreeThrows_Attempted'] = merged_df['LFTA_1'] + merged_df['WFTA_2']
merged_df['Opposing_Offensive_Rebounds'] = merged_df['LOR_1'] + merged_df['WOR_2']
merged_df['Opposing_Defensive_Rebounds'] = merged_df['LDR_1'] + merged_df['WDR_2']
merged_df['Opposing_Turnovers'] = merged_df['LTO_1'] + merged_df['WTO_2']

merged_df['TeamID'] = merged_df['WTeamID'].combine_first(merged_df['LTeamID'])

output = [
    'Season', 'Losses', 'Wins', 'TeamID', 'Points_Scored', 'FG_Made', 'FG_Atts', 
    'Threes_Made', 'Threes_Attempted', 'FreeThrows_Made', 
    'FreeThrows_Attempted', 'Offensive_Rebounds', 'Defensive_Rebounds', 
    'Assists', 'Turnovers', 'Points_Allowed', 'Opposing_FG_Made', 
    'Opposing_FG_Atts', 'Opposing_Threes_Made', 'Opposing_Threes_Attempted', 
    'Opposing_FreeThrows_Made', 'Opposing_FreeThrows_Attempted', 
    'Opposing_Offensive_Rebounds', 'Opposing_Defensive_Rebounds', 'Opposing_Turnovers'
]

team_summary_stats = merged_df.copy() 
team_summary_stats = team_summary_stats[output]

team_summary_stats['Total_Games'] = team_summary_stats['Wins'] + team_summary_stats['Losses']
team_summary_stats['Points_Per_Game'] = team_summary_stats['Points_Scored'] / team_summary_stats['Total_Games']
team_summary_stats['Win_Percentage'] = team_summary_stats['Wins'] / team_summary_stats['Total_Games']
team_summary_stats['FG_Percentage'] = (team_summary_stats['FG_Made'] / team_summary_stats['FG_Atts']) * 100
team_summary_stats['Threes_Per_Game'] = team_summary_stats['Threes_Attempted'] / team_summary_stats['Total_Games']
team_summary_stats['Turnovers_Per_Game'] = team_summary_stats['Turnovers'] / team_summary_stats['Total_Games']
team_summary_stats['Three_Point_Percentage'] = (team_summary_stats['Threes_Made'] / team_summary_stats['Threes_Attempted']) * 100
team_summary_stats['Free_Throws_Per_Game'] = team_summary_stats['FreeThrows_Attempted'] / team_summary_stats['Total_Games']
team_summary_stats['Free_Throw_Percentage'] = (team_summary_stats['FreeThrows_Made'] / team_summary_stats['FreeThrows_Attempted']) * 100
team_summary_stats['Offensive_Rebound_Rate'] = (team_summary_stats['Offensive_Rebounds'] / (team_summary_stats['Offensive_Rebounds'] + team_summary_stats['Opposing_Defensive_Rebounds'])) * 100
team_summary_stats['Defensive_Rebound_Rate'] = (team_summary_stats['Defensive_Rebounds'] / (team_summary_stats['Defensive_Rebounds'] + team_summary_stats['Opposing_Offensive_Rebounds'])) * 100
team_summary_stats['Opp_FG_Percentage'] = (team_summary_stats['Opposing_FG_Made'] / team_summary_stats['Opposing_FG_Atts']) * 100
team_summary_stats['Opp_Three_Point_Percentage'] = (team_summary_stats['Opposing_Threes_Made'] / team_summary_stats['Opposing_Threes_Attempted']) * 100
team_summary_stats['Opp_Free_Throws_Per_Game'] = team_summary_stats['Opposing_FreeThrows_Attempted'] / team_summary_stats['Total_Games']
team_summary_stats['Opp_Turnovers_Per_Game'] = team_summary_stats['Opposing_Turnovers'] / team_summary_stats['Total_Games']

team_summary_stats.fillna(0, inplace=True)

# Add team names to records so external data can be joined in 
team_summary_stats = team_summary_stats.merge(womens_teams, on=['TeamID'], how='left')

percentage_columns = [
    'Season', 'TeamName', 'TeamID', 'Win_Percentage', 'Points_Per_Game', 'FG_Percentage', 'Threes_Per_Game', 'Three_Point_Percentage', 
    'Free_Throws_Per_Game', 'Free_Throw_Percentage', 'Offensive_Rebound_Rate', 'Defensive_Rebound_Rate', 
    'Turnovers_Per_Game', 'Opp_FG_Percentage', 'Opp_Three_Point_Percentage', 'Opp_Free_Throws_Per_Game', 'Opp_Turnovers_Per_Game'
]

team_summary_stats = team_summary_stats[percentage_columns]

team_summary_stats.head(5) 

,Season,TeamName,TeamID,Win_Percentage,Points_Per_Game,FG_Percentage,Threes_Per_Game,Three_Point_Percentage,Free_Throws_Per_Game,Free_Throw_Percentage,Offensive_Rebound_Rate,Defensive_Rebound_Rate,Turnovers_Per_Game,Opp_FG_Percentage,Opp_Three_Point_Percentage,Opp_Free_Throws_Per_Game,Opp_Turnovers_Per_Game
0,2010,Air Force,3102.0,0.035714,51.821429,36.021505,16.071429,28.444444,13.214286,67.837838,34.316618,60.375276,17.714286,47.878404,37.630662,14.892857,14.392857
1,2010,Akron,3103.0,0.566667,62.233333,40.229192,12.466667,31.550802,19.600000,70.578231,39.664804,63.351254,18.266667,38.144330,31.868132,18.833333,18.700000
2,2010,Alabama,3104.0,0.379310,64.137931,39.811216,14.724138,29.508197,16.793103,61.601643,35.353535,63.567202,19.172414,39.736239,30.897704,21.965517,19.241379
3,2010,Alabama A&M,3105.0,0.518519,60.777778,40.058267,9.888889,30.711610,24.333333,69.863014,39.088983,60.614793,25.037037,39.600773,29.275362,23.518519,20.740741
4,2010,Alabama St,3106.0,0.413793,55.137931,34.808068,9.586207,28.057554,24.000000,64.798851,39.048474,66.666667,20.103448,35.306122,31.486880,26.655172,18.310345


In [5]:
### Calculate the Win Percentage for teams in the final 10 games of the season 


# Combine the winning and losing teams into a single DataFrame
win_data = womens_reg_season_data[['Season', 'DayNum', 'WTeamID']].copy()
win_data['IsWin'] = 1  # 1 for win

loss_data = womens_reg_season_data[['Season', 'DayNum', 'LTeamID']].copy()
loss_data['IsWin'] = 0  # 0 for loss

# Rename the columns to match for concatenation
win_data = win_data.rename(columns={'WTeamID': 'TeamID'})
loss_data = loss_data.rename(columns={'LTeamID': 'TeamID'})

# Combine the win and loss data
all_games = pd.concat([win_data, loss_data], ignore_index=True)

# Rank the games for each team (by DayNum, descending)
all_games['GameRank'] = all_games.groupby(['TeamID', 'Season'])['DayNum'].rank(method='first', ascending=False)

# Filter the last 10 games for each team
last_10_games = all_games[all_games['GameRank'] <= 10]

# Calculate wins and win percentage for each team in each season
win_percentage = last_10_games.groupby(['TeamID', 'Season']).agg(
    Wins=('IsWin', 'sum'),
    TotalGames=('IsWin', 'size')
).reset_index()

# Calculate win percentage
win_percentage['WinPercentage'] = (win_percentage['Wins'] / win_percentage['TotalGames'])
win_percentage = win_percentage.drop(columns=['Wins', 'TotalGames'])
win_percentage = win_percentage.rename(columns={'WinPercentage':'Win_pct_last_10_games'})

team_full_stats = team_summary_stats.merge(win_percentage, how='left', on=['Season','TeamID'])
team_full_stats.head(5) 

,Season,TeamName,TeamID,Win_Percentage,Points_Per_Game,FG_Percentage,Threes_Per_Game,Three_Point_Percentage,Free_Throws_Per_Game,Free_Throw_Percentage,Offensive_Rebound_Rate,Defensive_Rebound_Rate,Turnovers_Per_Game,Opp_FG_Percentage,Opp_Three_Point_Percentage,Opp_Free_Throws_Per_Game,Opp_Turnovers_Per_Game,Win_pct_last_10_games
0,2010,Air Force,3102.0,0.035714,51.821429,36.021505,16.071429,28.444444,13.214286,67.837838,34.316618,60.375276,17.714286,47.878404,37.630662,14.892857,14.392857,0.0
1,2010,Akron,3103.0,0.566667,62.233333,40.229192,12.466667,31.550802,19.600000,70.578231,39.664804,63.351254,18.266667,38.144330,31.868132,18.833333,18.700000,0.6
2,2010,Alabama,3104.0,0.379310,64.137931,39.811216,14.724138,29.508197,16.793103,61.601643,35.353535,63.567202,19.172414,39.736239,30.897704,21.965517,19.241379,0.3
3,2010,Alabama A&M,3105.0,0.518519,60.777778,40.058267,9.888889,30.711610,24.333333,69.863014,39.088983,60.614793,25.037037,39.600773,29.275362,23.518519,20.740741,0.6
4,2010,Alabama St,3106.0,0.413793,55.137931,34.808068,9.586207,28.057554,24.000000,64.798851,39.048474,66.666667,20.103448,35.306122,31.486880,26.655172,18.310345,0.5


In [6]:
output_dir = Path("../..") / "data" / "preprocessing"
output_dir.mkdir(parents=True, exist_ok =True)
output_path = output_dir / "womens_summary_season_data.csv"
team_full_stats.to_csv(output_path, index=False)